# 移動平均線的扣抵

## 程式實作：試算扣抵並預測20日移動平均線何時下彎

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import mplfinance as mpf
import numpy as np

In [2]:
def date_to_index(df_in,date_in) :
    number_array = df_in.index == date_in
    idx = 0
    for number_index in number_array :
        if number_index == True :
            break
        idx = idx + 1
    return idx

In [3]:
##### 使用2022年4月24日到2023年7月6日之日線圖資料 #####

# 載入從「好神通PLUS」輸出的Excel檔
df = pd.read_excel('Day20.xlsx')

# 保存K線的基本資訊(開、高、低、收、量)
df_k_line = df.drop(columns=['SMA20','MA5','MA10'])

# 保存價的移動平均線
df_sma = df.drop(columns=['開盤價','最高價','最低價','收盤價','成交量','MA5','MA10']) 

# 將K線的Columns的名稱由中文改為英文
df_k_line = df_k_line.rename(columns={'時間':'Date','開盤價':'Open','最高價':'High','最低價':'Low','收盤價':'Close','成交量':'Volume'})

# 將價的移動平均線的Columns的名稱由中文改為英文
df_sma = df_sma.rename(columns={'時間':'Date'})

# 設定今天日期
today_date = df_k_line.iloc[-1]['Date'].strftime("%Y-%m-%d")

# 將K線的Date設為Index
df_k_line.set_index(df_k_line['Date'],inplace=True)
df_k_line = df_k_line.drop(columns=['Date'])

# 將價的移動平均線的Date設為Index
df_sma.set_index(df_sma['Date'],inplace=True)
df_sma = df_sma.drop(columns=['Date'])

In [4]:
# 設定資料區間：區間的最後一天當作為「當前」的日期
df_k_line_range = df_k_line['2023-04-21':'2023-06-09']

# 「當前」的日期的股價之價位
current_price = df_k_line_range.iloc[-1]['Close']

# 設定20日移動平均線扣抵開始日期並取得其索引
deduction_date_idx = date_to_index(df_k_line_range,df_k_line_range.iloc[-20].name.strftime("%Y-%m-%d"))

# 取得總筆數
row, _ = df_k_line_range.shape

# 扣抵判斷
predict_days = 0
for idx in range(deduction_date_idx+1,row) :
    deduction_price = df_k_line_range.iloc[idx]['Close']
    if deduction_price < current_price  :
        print('扣小加大 ： {} 扣小({:.2f}) , 加大({:.2f})'.format(df_k_line_range.iloc[idx].name.strftime("%Y-%m-%d"),deduction_price,current_price))
    elif df_k_line_range.iloc[idx]['Close'] > current_price  :
        print('扣大加小 ： {} 扣大({:.2f}) , 加小({:.2f})'.format(df_k_line_range.iloc[idx].name.strftime("%Y-%m-%d"),deduction_price,current_price))
        # 以此案例，當出現「扣大加小」時跳出
        predict_days = predict_days + 1
        break
    predict_days = predict_days + 1
print('預計第{}天後移動平均線下彎'.format(predict_days))

扣小加大 ： 2023-05-16 扣小(43.60) , 加大(44.60)
扣小加大 ： 2023-05-17 扣小(43.95) , 加大(44.60)
扣小加大 ： 2023-05-18 扣小(44.40) , 加大(44.60)
扣大加小 ： 2023-05-19 扣大(45.15) , 加小(44.60)
預計第4天後移動平均線下彎


In [5]:
# 設定資料區間
df_sma_range = df_sma['2023-06-09':]
# 從「當前」的日期往後確認20日移動平均線的趨勢
relay_days = 0
for idx in range(1,len(df_sma_range)) :
    if df_sma_range.iloc[idx]['SMA20'] > df_sma_range.iloc[idx-1]['SMA20'] :
        print('{} ： 移動平均線上揚'.format(df_sma_range.iloc[idx].name.strftime("%Y-%m-%d")))
    elif df_sma_range.iloc[idx]['SMA20'] < df_sma_range.iloc[idx-1]['SMA20'] :
        print('{} ： 移動平均線下彎'.format(df_sma_range.iloc[idx].name.strftime("%Y-%m-%d")))
        # 以此案例，當出現「移動平均線下彎」時跳出
        relay_days = relay_days + 1
        break
    else :
        print('{} ： 移動平均線走平'.format(df_sma_range.iloc[idx].name.strftime("%Y-%m-%d")))
    relay_days = relay_days + 1
print('實際第{}天後移動平均線下彎'.format(relay_days))    

2023-06-12 ： 移動平均線上揚
2023-06-13 ： 移動平均線上揚
2023-06-14 ： 移動平均線上揚
2023-06-15 ： 移動平均線走平
2023-06-16 ： 移動平均線下彎
實際第5天後移動平均線下彎
